<a href="https://colab.research.google.com/github/osgeokr/GEE-PAM-Book/blob/main/cartoee_Google_Earth_Engine_%EA%B2%B0%EA%B3%BC%EB%A5%BC_%EC%9D%B8%EC%87%84%EC%9A%A9_%EC%A7%80%EB%8F%84%EB%A1%9C_%EC%83%9D%EC%84%B1%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cartoee: Google Earth Engine 결과를 인쇄용 지도로 생성하기

의존성 라이브러리인 [PROJ](https://proj.org/en/9.4/install.html)와 [GEOS](https://libgeos.org/)의 사전 설치 후, [cartopy](https://pypi.org/project/Cartopy/)와 [cartoee](https://pypi.org/project/cartoee/)를 이어서 설치합니다.

In [3]:
# PROJ와 GEOS 설치
!apt-get install libproj-dev proj-bin
!apt-get install libgeos-dev

# Cartopy와 cartoee 설치
!pip install -q -U cartopy cartoee

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libproj-dev is already the newest version (9.1.1-1~jammy0).
proj-bin is already the newest version (9.1.1-1~jammy0).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgeos-dev is already the newest version (3.11.1-1~jammy0).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 21.7 MB/s eta 0:00:00
  Using cached cartoee-0.0.5-py3-none-any.whl (10 kB)
  Using cached geos-0.2.3-py3-none-any.whl (400 kB)
  Using cached Cartopy-0.16.0.tar.gz (8.8 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem wi

# How to create publication quality maps using `cartoee`

`cartoee` is a lightweight module to aid in creating publication quality maps from Earth Engine processing results without having to download data. The `cartoee` package does this by requesting png images from EE results (which are usually good enough for visualization) and `cartopy` is used to create the plots. Utility functions are available to create plot aesthetics such as gridlines or color bars. **The notebook and the geemap cartoee module ([cartoee.py](https://geemap.org/cartoee)) were contributed by [Kel Markert](https://github.com/KMarkert). A huge thank you to him.**

In [4]:
%pylab inline

import ee
import geemap
import matplotlib.pyplot as plt

# import the cartoee functionality from geemap
from geemap import cartoee

Populating the interactive namespace from numpy and matplotlib


In [ ]:
geemap.ee_initialize()

## Plotting an image

In this first example we will explore the most basic functionality including plotting and image, adding a colorbar, and adding visual aethetic features. Here we will use SRTM data to plot global elevation.

In [ ]:
# get an image
srtm = ee.Image("CGIAR/SRTM90_V4")

In [ ]:
# geospatial region in format [E,S,W,N]
region = [180, -60, -180, 85]  # define bounding box to request data
vis = {"min": 0, "max": 3000}  # define visualization parameters for image

In [ ]:
fig = plt.figure(figsize=(15, 10))

# use cartoee to get a map
ax = cartoee.get_map(srtm, region=region, vis_params=vis)

# add a colorbar to the map using the visualization params we passed to the map
cartoee.add_colorbar(ax, vis, loc="bottom", label="Elevation", orientation="horizontal")

# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[60, 30], linestyle=":")

# add coastlines using the cartopy api
ax.coastlines(color="red")

plt.show()

This is a decent map for minimal amount of code. But we can also easily use matplotlib colormaps to visualize our EE results to add more color. Here we add a `cmap` keyword to the `.get_map()` and `.add_colorbar()` functions.

In [ ]:
fig = plt.figure(figsize=(15, 10))

cmap = "gist_earth"  # colormap we want to use
# cmap = "terrain"

# use cartoee to get a map
ax = cartoee.get_map(srtm, region=region, vis_params=vis, cmap=cmap)

# add a colorbar to the map using the visualization params we passed to the map
cartoee.add_colorbar(
    ax, vis, cmap=cmap, loc="right", label="Elevation", orientation="vertical"
)

# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[60, 30], linestyle="--")

# add coastlines using the cartopy api
ax.coastlines(color="red")

ax.set_title(label="Global Elevation Map", fontsize=15)

plt.show()

## Plotting an RGB image

`cartoee` also allows for plotting of RGB image results directly. Here is an example of plotting a Landsat false-color scene.

In [ ]:
# get a landsat image to visualize
image = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_044034_20140318")

# define the visualization parameters to view
vis = {"bands": ["B5", "B4", "B3"], "min": 0, "max": 5000, "gamma": 1.3}

In [ ]:
fig = plt.figure(figsize=(15, 10))

# use cartoee to get a map
ax = cartoee.get_map(image, vis_params=vis)

# pad the view for some visual appeal
cartoee.pad_view(ax)

# add the gridlines and specify that the xtick labels be rotated 45 degrees
cartoee.add_gridlines(ax, interval=0.5, xtick_rotation=45, linestyle=":")

# add the coastline
ax.coastlines(color="yellow")

plt.show()

By default, if a region is not provided via the `region` keyword the whole extent of the image will be plotted as seen in the previous Landsat example. We can also zoom to a specific region of an image by defining the region to plot.

In [ ]:
fig = plt.figure(figsize=(15, 10))

# here is the bounding box of the map extent we want to use
# formatted a [E,S,W,N]
zoom_region = [-121.8025, 37.3458, -122.6265, 37.9178]

# plot the map over the region of interest
ax = cartoee.get_map(image, vis_params=vis, region=zoom_region)

# add the gridlines and specify that the xtick labels be rotated 45 degrees
cartoee.add_gridlines(ax, interval=0.15, xtick_rotation=45, linestyle=":")

# add coastline
ax.coastlines(color="yellow")

plt.show()

## Adding north arrow and scale bar

In [ ]:
fig = plt.figure(figsize=(15, 10))

# here is the bounding box of the map extent we want to use
# formatted a [E,S,W,N]
zoom_region = [-121.8025, 37.3458, -122.6265, 37.9178]

# plot the map over the region of interest
ax = cartoee.get_map(image, vis_params=vis, region=zoom_region)

# add the gridlines and specify that the xtick labels be rotated 45 degrees
cartoee.add_gridlines(ax, interval=0.15, xtick_rotation=45, linestyle=":")

# add coastline
ax.coastlines(color="yellow")

# add north arrow
cartoee.add_north_arrow(
    ax, text="N", xy=(0.05, 0.25), text_color="white", arrow_color="white", fontsize=20
)

# add scale bar
cartoee.add_scale_bar_lite(
    ax, length=10, xy=(0.1, 0.05), fontsize=20, color="white", unit="km"
)

ax.set_title(label="Landsat False Color Composite (Band 5/4/3)", fontsize=15)

plt.show()